In [ ]:
# if the bayesm package is not installed, use install.packages("bayesm") to
# install the package first
library(bayesm)

data(camera)

In [ ]:
#' Simulates market shares based on the multinomial logit model. The preference
#' parameters are assumed to be estimated using a Bayesian hierarchical model.
#' The number of draws of the population level parameters are determined by the
#' MCMC chain length of the provided estimation.
#' @param X matrix of product attributes in a customer's consideration sets
#' @param estimation object storing the estimation output from the choice model by
#' `rhierMnlRwMixture()` in the `bayesm` package.
#' @param ndraws number of beta draws in the market share simulation. More draws
#' obviously take longer to run but yield more precise market share prediction.
#' @param agg default `TRUE`. If `TRUE`, aggregate market shares over
#' parameter distributions. Otherwise report the collection of market shares.
#' @return the average market shares of all market share simulations
mktsharesim = function(X, estimation, ndraw=200, agg=TRUE) {
    # `bayesm.riherMnlRwMixture()` has some quirky output data structure
    n_attr = length(estimation$nmix$compdraw[[1]][[1]]$mu)
    n_opt = nrow(X)
    n_comp = length(estimation$nmix$compdraw[[1]])
    R = length(estimation$nmix$compdraw)
    burn = R / 2
    shares = matrix(0.0, n_opt, R - burn)

    # counter for market shares. Indexing for MCMC chain does not start at 1 due
    # to burning
    i = 1
    # each column is a MCMC draw. Transformed to work better with R's 
    # column-major design
    comp_probs = t(estimation$nmix$probdraw)
    for (r in (burn + 1):R) {
      pvec = comp_probs[, r]
      comp_draws = sample.int(n_comp, ndraw, TRUE, pvec)
      # comp_frequency = table(comp_draws)
      beta = matrix(0.0, n_attr, ndraw)
      # used for allocating beta draws
      beta_index = 0
      for (comp in 1:n_comp) {
        n_customer_comp = sum(comp_draws == comp)
        comp_start = beta_index + 1
        comp_end = beta_index + n_customer_comp
        mu = estimation$nmix$compdraw[[r]][[comp]]$mu
        rooti = estimation$nmix$compdraw[[r]][[comp]]$rooti
        noise = t(solve(rooti)) %*% matrix(rnorm(n_attr * n_customer_comp), n_attr)
        beta[, comp_start:comp_end] = mu + noise
        # enforce the negative sign restriction for the parameter on price
        beta[n_attr, comp_start:comp_end] =
          -exp(beta[n_attr, comp_start:comp_end])
        beta_index = beta_index + n_customer_comp
      }
      utilities = exp(X %*% beta)
      shares_mat = utilities / matrix(rep(colSums(utilities), each=n_opt),
          n_opt)
      shares[, i] = rowMeans(shares_mat)
      i = i + 1
    }
    if (agg) {
      return(rowMeans(shares))
    } else {
      return(shares)
    }
}

# conjoing estimation
nvar = ncol(camera[[1]]$X)

Data = list(
  p = 5,
  lgtdata = camera
)
Prior = list(
  ncomp = 1,
  SignRes = c(double(nvar - 1), -1)
)
Mcmc = list(
  R = 30000,
  keep = 15,
  nprint = 1000
)

# estimate logit model
set.seed(777)
estimation = rhierMnlRwMixture(Data, Prior, Mcmc)


In [ ]:
# a)
# create a price sequence
price <- seq(from = 0.5, to = 5, by = 0.1)
# print only the first 10 values
print(price[1:10])


In [ ]:
# b)
# create a profit value with same length of price
profit <- seq(from = 0.5, to = 5, by = 0.1)
# input seed
set.seed(1012)
# loop
for (i in 1:length(profit))
  {
  canon <- c(1, 0, 0, 0, 1, 0, 1, 1, 0, price[i])
  sony <-c (0, 1, 0, 0, 0, 0, 0, 0, 0, 2)
  nikon <- c(0, 0, 1, 0, 0, 0, 0, 0, 0, 2)
  panasonic <- c(0, 0, 0, 1, 0, 0, 0, 0, 0, 2)
  none <- c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
  brand <- rbind(canon, sony, nikon, panasonic, none)
  pred <- mktsharesim(brand, estimation, ndraw = 200, agg = TRUE)
  
  # take only first line of profit is our estimate value
  pred <- pred[1]
  final <- (price[i] - 0.5) * pred * 100
  profit[i] <- final
  }

print(profit)
print(final)

In [ ]:
#c)
library(ggplot2)
# create matrix
m_pp <- cbind(price, profit)

# make the matrix into a data frame
m_pp <- data.frame(m_pp)
m_pp
#plot the line chart according to the data frame
ggplot(data = m_pp, aes(price, profit)) +
geom_line() + geom_vline(xintercept = price[profit == max(profit)],
color = '#cd9ec68a')

max(profit)
price[profit == max(profit)]

# The profit maximizing price is $200


In [ ]:
# d)
# create a profit value with same length of price
profit_1 <- seq(from = 0.5, to = 5, by = 0.1)
# input seed
set.seed(1026)
# loop
for (i in 1:length(profit_1))
  {
  Sony <- c(0, 1, 0, 0, 0, 0, 0, 0, 0, 2)
  Nikon <- c(0, 0, 1, 0, 0, 0, 0, 0, 0, 2)
  Panasonic <- c(0, 0, 0, 1, 0, 0, 0, 0, 0, 2)
  None <- c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
  Canon_wifi <- c(1, 0, 0, 0, 1, 0, 1, 1, 1, price[i])
  brand_wifi <- rbind(Canon_wifi, Sony, Nikon, Panasonic, None)
  pred2 <- mktsharesim(brand_wifi, estimation, ndraw = 200, agg = TRUE)
  # take only first line of profit is our estimate value
  pred2 <- pred2[1]
  final_wifi <- (price[i] - 0.55) * pred2 * 100
  profit_1[i] <- final_wifi
}

print(profit_1)
print(final_wifi)

m_pp1 <- cbind(price, profit_1)

# make the matrix into a data frame
m_pp1 <- data.frame(m_pp)
m_pp1
#plot the line chart according to the data frame
ggplot(data = m_pp1, aes(price, profit_1)) +
  geom_line() + geom_vline(xintercept = price[profit_1 == max(profit_1)],
                           color = '#2222dc8a')

max(profit_1)
price[profit_1 == max(profit_1)]

# the max profit is 67.96 > 53.5
# therefore they should change, and set price at 250